In [1]:
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage

load_dotenv()

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
ai_message = model.invoke([HumanMessage(content="Hi! I'm Dan.")])

print(ai_message.content)

Hi Dan! It's great to meet you. How can I help you today?


In [2]:
ai_message = model.invoke([
    HumanMessage(content="Hi! I'm Dan."),
    AIMessage(content="Hi Dan! It's great to meet you. How can I help you today?"),
    HumanMessage(content="What's my name?"),
])

print(ai_message.content)

Your name is Dan!


In [10]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            "You talk like a pirate."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [11]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [3]:
config = { "configurable": {"thread_id": "abc123"}}

In [12]:
output = app.invoke({"messages": [HumanMessage(content="Hi! I'm Dan.")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Dan! A fine name for a landlubber, or perhaps a future scallywag o' the seas! What brings ye to these here waters?


In [13]:
output = app.invoke({"messages": [HumanMessage(content="What's my name?")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Blast me barnacles! Ye just told ol' Cap'n Craggybeard yer name, ye did! Ye be **Dan**, if me memory serves me right and true! Did ye take a knock to the head, or be ye just testin' me wits, eh?


In [14]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            "You are a helpful assistant. Answer all the questions to the best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "abc456"}}

output = app.invoke(
    {"messages": [HumanMessage(content="Hi! I'm Dan.")], "language": "German"}, config
)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hallo, Dan!


In [22]:
output = app.invoke({"messages": [HumanMessage(content="What's my name?")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Dein Name ist Dan.
